In [30]:
import pyvis as pv
import networkx as nx
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import random as rd

In [69]:
# Importando as áreas
raw_data = pd.read_csv("c:/Users/Usuario/Documents/Estudos/Data Science e Programação/Datasets/colaboradores.csv")
raw_data = raw_data[['nome', 'area']]

In [51]:
# Criando uma lista de quantidade de pessoas por área
areas = set(raw_data['area'])

In [52]:
# Criando uma função para encontrar a estimativa do número de conecções 
def total_conections(name):
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    total = len(raw_data.loc[raw_data['area'] == area])
    conections = rd.randint(round(total*0.1), round(total*1.5))
    return conections

In [67]:
# Criando uma função para atribuir a ponderação de cada nome
def weight_definition(name):
    # Encontrando a área
    areas_dict = dict.fromkeys(areas, 1) 
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    # Atribuindo o peso adequado
    areas_dict[area] = 100
    raw_data['probabilidade'] = raw_data.apply(lambda x: areas_dict[x['area']], axis = 1)
    # Criando o vetor de itens e pesos
    itens = list(raw_data['nome'])
    #probability = list(raw_data['probabilidade'])
    #weights = [j/sum(probability) for j in probability]
    weights = list(raw_data['probabilidade'])
    return itens, weights

In [55]:
# Criando uma função para criar as interações de cada pessoa
def ona_conections(name):
    # Lista completa de nomes
    total_names = raw_data['nome']
    total_names.remove(i)
    # Total de conecções
    total_conections = total_conections(i)
    # Encontrando a área
    areas_dict = dict.fromkeys(areas, 1) 
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    # Atribuindo o peso adequado
    areas_dict[area] = 10
    weights = [j/sum(areas_dict.values()) for j in areas_dict.values()]
    

In [36]:
#for i in list(raw_data['nome']):  